# Lab 6 - Checksum Recalculation and Packet Deparsing

This lab describes how to recompute the checksum of a header. Recomputing the checksum is necessary if the packet header was modified by the P4 program. The lab also describes how a P4 program performs deparsing to emit headers. 

<img src="./labs_files/lab1/figs/fabric_topology.png" width="550px"><br>

# Background

## Introduction to checksums

Several protocols use checksums to validate the integrity of the packet headers. A checksum is a small value derived from another data block, often through a checksum algorithm such as the Cyclic Redundancy Check (CRC). The checksum calculation and verification start with the sender calculating the checksum of the data before transmitting the packet. Then, the checksum value is inserted into the packet header. Upon receiving the packet, the receiver computes the checksum of the received packet using the same algorithm as the one used by the sender. If the calculated checksum value matches the one in the packet header, the packet is verified; otherwise, a transmission error has occurred. Incorrect checksums typically lead to dropping the packet by the switch.


### Checksums in P4

In a P4 program, the developer may change the packet headers. For example, if the program is implementing a routing function, then header fields such as the Time-to-live (TTL) must be modified. Any change to the header fields will cause the checksum value to change. Therefore, it is necessary to recompute the checksum in the P4 program in case modifications are made to the header fields. 

<img src="./labs_files/lab6/figs/snippet.png" width="450px"><br>

The syntax for updating the checksum in P4 (V1Model) is as follows:

    update_checksum(condition, data, checksum_output, algorithm)

<ul>
<li>condition: a condition that is evaluated before updating the checksum. If the condition is true, the checksum is updated. Otherwise, the checksum remains as it is in the packet. Here we often check if the header is valid (i.e., it was parsed or set to be valid by the programmer). For example, in Figure 1, the IPv4 header is checked if valid.</li>
<li>data: the data whose checksum is to be computed. This typically includes the header fields of the protocol which uses the checksum. The example above shows the header fields of IPv4.</li>
<li>checksum_output: the parameter that the checksum will be written to once it has been computed. In the example above, we are writing the resulting checksum value to the hdrChecksum field of IPv4.</li>
<li>algorithm: the algorithm used by the protocol to compute the checksum. For example, for IPv4, the IETF RFC 7911 state that the checksum field is the 16-bit one’s complement of the one’s complement sum of all 16-bit words in the header. This checksum is implemented in the V1Model using the HashAlgorithm.csum16 hash function. </li>
</ul>

It is also possible in P4 to verify the checksum. The V1Model provides the checksum verification extern function verify_checksum which sets the checksum_error bit in the standard metadata in case the verification fails, causing the packet to be dropped. The syntax for the verify_checksum is the same as that of update_checksum described above.

    verify_checksum(condition, data, checksum_output, algorithm)
    
## Deparsing

The P4 program includes a deparser that specifies which headers are to be emitted. The deparser emits the headers and the payload of the original packet. Note that only the valid headers are emitted. A header is considered valid after it has been parsed in the P4 program or after the program explicitly validates the header with the function setValid(). 

The deparser is defined as a control block and is executed after finishing the packet processing by the other control blocks. Consider Figure 2. The deparser has a packet_out type in its parameters. The packet_out type includes the emit method which accepts the headers to be reassembled when the deparser constructs the outgoing packet. Note that the order of emitting packets' headers is important, and the headers are only emitted in case they are valid.

<img src="./labs_files/lab6/figs/deparsing.png" width="450px"><br>


# Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

# Step 2: Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Create the Experiment Slice

The following creates three node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice with the name "lab6"

In [ ]:
slice = fablib.new_slice(name="lab6")

### Step 3.2: Define the sites
The code below requests three sites from FABRIC: MICH, STAR, and NCSA

<img src="./labs_files/lab1/figs/fabric_sites.png" width="550px"><br>

In [ ]:
site1='MICH'
site2='STAR'
site3='NCSA'

print (f'The selected sites are {site1}, {site2}, {site3}') 

### Step 3.3: Creating the nodes
The code below creates three nodes: server1, switch, and server2. The servers (server1 and server2) use the following
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1 and server3 will be created in site3

<img src="./labs_files/lab1/figs/creating_nodes.PNG" width="550px"><br>

In [ ]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server2 = slice.add_node(name="server2", 
                      site=site3, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Adding the interfaces to the servers
The code below adds a Network Interface Card (NIC) to each server.

<img src="./labs_files/lab1/figs/adding_nics.PNG" width="550px"><br>

In [ ]:
server1_iface = server1.add_component(model='NIC_Basic').get_interfaces()[0]
server2_iface = server2.add_component(model='NIC_Basic').get_interfaces()[0]

### Step 3.5: Creating a node for the P4 switch
The code below creates a node that will run the P4 switch. The node use the following
<ul>
    <li> 16 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 40GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

The node will be created in site2

<img src="./labs_files/lab1/figs/adding_switch.PNG" width="550px"><br>

In [ ]:
# Add a node
switch = slice.add_node(name="switch", 
                      site=site2, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

### Step 3.6: Adding two interfaces to the switch
The code below adds two Network Interface Cards (NICs) to the switch.

<img src="./labs_files/lab1/figs/adding_switch_ports.PNG" width="550px"><br>

In [ ]:
switch_iface1 = switch.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch_iface2 = switch.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]

### Step 3.7: Connecting site1 and site2
Create a site-to-site network between site1 and site2 connecting server1 and the P4 switch

<img src="./labs_files/lab1/figs/connecting_nodes_server1_switch.PNG" width="550px"><br>

In [ ]:
net1 = slice.add_l2network(name='net1', interfaces=[server1_iface, switch_iface1])

### Step 3.8: Connecting site2 and site3
Create a site-to-site network between site2 and site3 connecting the P4 switch and server2

<img src="./labs_files/lab1/figs/connecting_nodes_server2_switch.PNG" width="550px"><br>

In [ ]:
net2 = slice.add_l2network(name='net2', interfaces=[switch_iface2, server2_iface])

### Step 3.9: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [ ]:
#Submit Slice Request
slice.submit();

# Step 4: Installing the required packages
In this step, we will install the required packages to run the labs. Specifically, we will install the BMv2 software switch and its control plane, the P4 compiler (p4c), and net-tools.


## Step 4.1 Installing BMv2
The BMv2 software switch will be installed on the switch node. We will upload the script [scripts/install_bmv2.sh](./scripts/install_bmv2.sh) to the switch and execute it

In [ ]:
switch = slice.get_node(name="switch")     
switch.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

## Step 4.2 Installing net-tools
The net-tools package will be installed on the switch, server1 and server2 nodes. This package will allow us to use the ifconfig and the arp commands 

In [ ]:
server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")
stdout, stderr = server1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch.execute(f'sudo apt-get install -y net-tools', quiet=True)

## Step 4.3 Installing scapy
Installing scapy to be able to craft and send raw packets on the servers


In [ ]:
stdout, stderr = server1.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)

# Step 5: Assigning IP and MAC addresses
In this step, we will assign IPv4 addresses to the interfaces of the servers and the switch. We will also hardcode the MAC addresses. 

## Step 5.1: Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

<img src="./labs_files/lab1/figs/interfaces.PNG" width="550px"><br>

In [ ]:
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()
print(f'server1_iface: {server1_iface_name}')

node2_iface = server2.get_interface(network_name='net2') 
server2_iface_name = node2_iface.get_device_name()
print(f'server2_iface: {server2_iface_name}')

switch_iface1 = switch.get_interface(network_name='net1') 
switch_iface1_name = switch_iface1.get_device_name()
print(f'switch_iface1: {switch_iface1_name}')

switch_iface2 = switch.get_interface(network_name='net2') 
switch_iface2_name = switch_iface2.get_device_name()
print(f'switch_iface2: {switch_iface2_name}')

## Step 5.2: Turning all interfaces up
In this step, we will use the ip link command to turn the interfaces up

<img src="./labs_files/lab1/figs/interfaces_up.PNG" width="550px"><br>

In [ ]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface1_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface2_name} up', quiet=True)

## Step 5.3: Hardcode MAC addresses
For simplicity, we will use the following MAC addresses for the interfaces:
<ul>
    <li> server1_iface_MAC = '00:00:00:00:00:01' (shown as 00:01 in the figure below) </li>
    <li>switch_iface1_MAC = '00:00:00:00:00:02' (shown as 00:02 in the figure below)</li>
    <li>switch_iface2_MAC = '00:00:00:00:00:03' (shown as 00:03 in the figure below)</li>
    <li>server2_iface_MAC = '00:00:00:00:00:04' (shown as 00:04 in the figure below)</li>
</ul>

<img src="./labs_files/lab1/figs/mac_addresses.PNG" width="550px"><br>

In [ ]:
server1_iface_MAC = '00:00:00:00:00:01'
switch_iface1_MAC = '00:00:00:00:00:02'
switch_iface2_MAC = '00:00:00:00:00:03'
server2_iface_MAC = '00:00:00:00:00:04'

## Step 5.4 Configuring the IP and MAC addresses on server1_iface and switch_iface1

We will use the network 192.168.1.0/24 between Site1 and Site2. We will assign the IP address 192.168.1.10 to server1's interface and 192.168.1.1 to its neighboring interface on the switch.

<img src="./labs_files/lab1/figs/IPs_1.PNG" width="550px"><br>

In [ ]:
server1 = slice.get_node(name="server1")     

server1_switch_subnet = "192.168.1.0/24"
server1_ip = '192.168.1.10/24'
switch_ip1 = '192.168.1.1/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} {switch_ip1}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} hw ether {switch_iface1_MAC}')

## Step 5.5: Configuring the IP and MAC addresses on switch_iface2 and server2_iface

We will use the network 192.168.2.0/24 between Site2 and Site3. We will assign the IP address 192.168.2.10 to server2's interface and 192.168.2.1 to its neighboring interface on the switch.

<img src="./labs_files/lab1/figs/IPs_2.PNG" width="550px"><br>

In [ ]:
server2 = slice.get_node(name="server2")     

server2_switch_subnet = "192.168.2.0/24"
server2_ip = '192.168.2.10/24'
switch_ip2 = '192.168.2.1/24'

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} {server2_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} {switch_ip2}')

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} hw ether {server2_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} hw ether {switch_iface2_MAC}')

# Step 6: Configure forwarding and routing

## Step 6.1: Enable forwarding on the switch

The command "sudo sysctl -w net.ipv4.ip_forward=1" is used to enable IP forwarding on a Linux system.

IP forwarding is a feature that allows a system to act as a router by forwarding network packets from one network interface to another. By default, IP forwarding is usually disabled on Linux systems for security reasons. 

The command will be executed on the switch device.

In [ ]:
command = 'sudo sysctl -w net.ipv4.ip_forward=1' 
stdout, stderr = switch.execute(command, quiet=True)

## Step 6.2: Delete routing entries for the routes to force traffic to go through the BMv2 switch

In this step, we are deleting the routes on the switch's routing table in Linux. By deleting the routes, the packets will go through the BMv2 switch instead of being forwarded by the kernel

In [ ]:
stdout, stderr = switch.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)

## Step 6.3: Configure routing

In this step, we will configure static routes on server1 and server2. 
<ul>
    <li> For server1, we will add a route to reach the network 192.168.2.0/24 via 192.168.1.1 </li>
    <li> For server2, we will add a route to reach the network 192.168.1.0/24 via 192.168.2.1 </li>
</ul>

<img src="./labs_files/lab1/figs/routing.PNG" width="550px"><br>

In [ ]:
gw1 = switch_ip1.split('/')[0]
gw2 = switch_ip2.split('/')[0]
stdout, stderr = server1.execute(f'sudo ip route add {server2_switch_subnet} via {gw1}')
stdout, stderr = server2.execute(f'sudo ip route add {server1_switch_subnet} via {gw2}')

## Step 6.4: Configure ARP

In this step, we will configure static ARP entries on server1 and server2. The reason we are doing this is because the switch does not process ARP packets unless programmed to. To make sure that ARP packets are not sent towards the switch, we will hardcode the MACs on the servers.

For each server, we will add an ARP entry to its switch's neighboring interface.

In [ ]:
stdout, stderr = server1.execute(f'sudo arp -s {gw1} {switch_iface1_MAC}')
stdout, stderr = server2.execute(f'sudo arp -s {gw2} {switch_iface2_MAC}')

# Step 7: Implementing checksum calculation in P4
   
This section demonstrates how to update the checksum of an IPv4 packet after being modified by the P4 program. We will be using the P4 program that implements the routing function.  

# Step 7.1: Inspecting the P4 code

Inspect the content of the [ingress.p4](./labs_files/lab6/src/ingress.p4) file before implementing the checksum calculation. 

<img src="./labs_files/lab6/figs/ingress.png" width="550px"><br>

Note how the action forward is modifying the TTL value in the IPv4 header. Since the program is modifying the header fields, it is necessary to recompute and update the checksum of the header. 

<hr>

Inspect the content of the [checksum.p4](./labs_files/lab6/src/checksum.p4) file. 

<img src="./labs_files/lab6/figs/checksum.png" width="550px"><br>

The figure above shows that two empty control blocks exist in the checksum.p4 file. The first control MyVerifyChecksum verifies the checksum for the packet. The second control block MyComputeChecksum updates the checksum of the packet. We will only focus for now on computing the checksum. The upcoming steps show what happens when the checksum is not updated after modifying the IPv4 header (i.e., decrementing the TTL).



# Step 8: Uploading and running the P4 program on the switch

In this step, we upload the P4 program to the switch, compile it, and start the switch daemon. 

## Step 8.1: Uploading the P4 program

The P4 program [basic.p4](labs_files/lab6/src/basic.p4) is located under lab_files/lab6/src.

We will be uploading the whole directory since it includes other P4 files. 

In [ ]:
switch = slice.get_node(name='switch')        
switch.upload_directory('labs_files/lab6/src', '/home/ubuntu/lab6')

## Step 8.2: Compiling the P4 program

In this step, we will use the p4c compiler to compile the program.

Launch a new terminal by clicking on "File" -> "New" -> "Terminal".

<img src="./labs_files/lab6/figs/terminal.gif" width="600px"><br>


Copy the output of the command below and paste into the terminal to enter to the switch.

In [ ]:
switch.get_ssh_command()

Run the following commands in the terminal:
    
    sudo su
    p4c lab6/src/basic.p4
    simple_switch -i 0@ens8 -i 1@ens7 basic.json --log-console
    
<img src="./labs_files/lab6/figs/daemon.png" width="750px"><br>

## Step 8.3: Populating table from the control plane

In this step we will populate the forwarding table by executing a script. We will learn how to populate the tables manually in another lab.

In [ ]:
switch.upload_file('labs_files/lab6/rules.sh', 'rules.sh')
stdout, stderr = switch.execute('chmod +x rules.sh && ./rules.sh')

## Step 8.4: Starting tcpdump on server2 

In this step, we will start tcpdump on server2 to inspect the packets arriving at its interface.

Launch a new terminal by clicking on "File" -> "New" -> "Terminal".

<img src="./labs_files/lab6/figs/terminal.gif" width="600px"><br>


Copy the output of the command below and paste into the terminal to enter to the switch.

In [ ]:
server2.get_ssh_command()

Type the following command in the terminal:

    sudo tcpdump -v -i ens7
    
<img src="./labs_files/lab6/figs/tcpdump.png" width="600px"><br>

## Step 8.5: Sending a packet without checksum update

In this step, we will send a packet to 192.168.2.20. The switch does not implement checksum update. 


In [ ]:
server1.upload_file('labs_files/lab6/src/send.py', 'send.py')
stdout, stderr = server1.execute(f'sudo python3 send.py ens7 192.168.1.10 192.168.2.20 HelloWorld')

## Step 8.6: Inspect the logs on server2

Go back to the terminal of server2 to inspect the output of tcpdump.

<img src="./labs_files/lab6/figs/bad_checksum.png" width="800px"><br>

We can see that the checksum in that packet is incorrect. This is because the P4 program changed the header field value (i.e., TTL), but did not update the checksum in the packet. 

## Step 8.7: Updating the P4 code to compute the checksum

Edit the content of the [checksum.p4](./labs_files/lab6/src/checksum.p4) file and add the following:

    update_checksum(
	    hdr.ipv4.isValid(),
        { 
            hdr.ipv4.version,
	        hdr.ipv4.ihl,
            hdr.ipv4.diffserv,
            hdr.ipv4.totalLen,
            hdr.ipv4.identification,
            hdr.ipv4.flags,
            hdr.ipv4.fragOffset,
            hdr.ipv4.ttl,
            hdr.ipv4.protocol,
            hdr.ipv4.srcAddr,
            hdr.ipv4.dstAddr 
         },
         hdr.ipv4.hdrChecksum,
         HashAlgorithm.csum16
     );

<img src="./labs_files/lab6/figs/update_checksum.png" width="550px"><br>

**Press Ctrl + s to save the changes.**

## Step 8.8: Uploading the P4 program

The P4 program [basic.p4](labs_files/lab6/src/basic.p4) is located under lab_files/lab6/src.

We will be uploading the whole directory since it includes other P4 files. 

In [ ]:
switch = slice.get_node(name='switch')        
switch.upload_directory('labs_files/lab6/src', '/home/ubuntu/lab6')
stdout, stderr = switch.execute('sudo pkill switch')

Go back to the terminal of the switch and issue the following commands to compile and run the P4 program:

    p4c lab6/src/basic.p4
    simple_switch -i 0@ens8 -i 1@ens7 basic.json --log-console
    
<img src="./labs_files/lab6/figs/p4c.png" width="800px"><br>

## Step 8.9: Populating table from the control plane

In this step we will populate the forwarding table by executing a script. We will learn how to populate the tables manually in another lab.

In [ ]:
switch.upload_file('labs_files/lab6/rules.sh', 'rules.sh')
stdout, stderr = switch.execute('chmod +x rules.sh && ./rules.sh')

## Step 8.10: Sending a packet with checksum update

In this step, we will send a packet to 192.168.2.20. The switch updates the checksum. 


In [ ]:
server1.upload_file('labs_files/lab6/src/send.py', 'send.py')
stdout, stderr = server1.execute(f'sudo python3 send.py ens7 192.168.1.10 192.168.2.20 HelloWorld')

## Step 8.11: Inspect the logs on server2

Go back to the terminal of server2 to inspect the output of tcpdump.

<img src="./labs_files/lab6/figs/good_checksum.png" width="750px"><br>

We can see that the checksum in that packet is now correct (tcpdump did not complain about the correctness of the checksum). This is because the P4 program updated the checksum in the IPv4 header after changing the header field value (i.e., TTL). 


# Step 9: Updating the deparser in the P4 code

In this section we will update the code of the deparser so that the IPv4 header is not emitted. We will then verify this operation using Wireshark. 


## Step 9.1: Updating the deparser in the P4 code

Edit the content of the [deparser.p4](./labs_files/lab6/src/deparser.p4) file. 

We can see in the figure above that the deparser is emitting both the Ethernet header and the IPv4 header. We verified in the previous step that Wireshark was able to recognize both headers.


Remove the *packet.emit(hdr.ipv4);* line to emit only the Ethernet header and save the file:


<img src="./labs_files/lab6/figs/remove_ipv4.png" width="550px"><br>

**Press Ctrl + s to save the changes.**


## Step 9.2: Uploading the P4 program

The P4 program [basic.p4](labs_files/lab6/src/basic.p4) is located under lab_files/lab6/src.

We will be uploading the whole directory since it includes other P4 files. 

In [ ]:
switch = slice.get_node(name='switch')        
switch.upload_directory('labs_files/lab6/src', '/home/ubuntu/lab6')
stdout, stderr = switch.execute('sudo pkill switch')

Go back to the terminal of the switch and issue the following commands to compile and run the P4 program:

    p4c lab6/src/basic.p4
    simple_switch -i 0@ens8 -i 1@ens7 basic.json --log-console
    
<img src="./labs_files/lab6/figs/p4c.png" width="750px"><br>

## Step 9.3: Populating table from the control plane

In this step we will populate the forwarding table by executing a script. We will learn how to populate the tables manually in another lab.

In [ ]:
switch.upload_file('labs_files/lab6/rules.sh', 'rules.sh')
stdout, stderr = switch.execute('chmod +x rules.sh && ./rules.sh')

## Step 9.4: Sending a packet

In this step, we will send a packet to 192.168.2.20.


In [ ]:
server1.upload_file('labs_files/lab6/src/send.py', 'send.py')
stdout, stderr = server1.execute(f'sudo python3 send.py ens7 192.168.1.10 192.168.2.20 HelloWorld')

## Step 9.5: Inspect the logs on server2

Go back to the terminal of server2 to inspect the output of tcpdump.

<img src="./labs_files/lab6/figs/bogus_packet.png" width="570px"><br>

We can see that the output is different from the previous tcpdump log. The packet is bogus/corrupt since the IPv4 header is not emitted. 